## Test the Different Components of the ML Data Pipeline 

In [1]:
import pathlib,os,sys
_base_module_path = '/home/monte.flora/python_packages'
modules = ['wofs_ml_severe']
paths = [sys.path.append(os.path.join(_base_module_path, m)) for m in modules]

### 0. Sending an email

### 1. Ensemble Storm Track Creation 


### 2. Extracting ML Features 

Create Unit test! 

Adding new ML features 
1. Ensemble maximum for both spatial and amplitude features [X]
2. Conditional amplitude ensemble statistics (i.e., only including ensemble members within a storm)[X]
3. Number of ensemble members within an ensemble storm track [X] 
4. Ensemble avg. of the area of individual storm tracks within an ensemble storm tracks (call it P). Ratio of P to the full ensemble storm area area. Idea: ensemble storm track can be large due to the lack of overlap [X]. 
7. Dummy variable (binary) if a storm is elevated. Idea: minimum height of dBZ exceeding a threshold [X]
8. Okubo-Weiss treated as a minimum variable [X]

In [17]:
from wofs_ml_severe.data_pipeline.ml_data_generator import MLDataGenerator
from wofs_ml_severe.data_pipeline.ensemble_track_segmentation import generate_ensemble_track_file
import numpy as np
import xarray as xr
def test_feature_extract():
        NV, NE, NY, NX = (3,5,300,300)
        
        # Generate the track file. 
        file_30M = '/home/monte.flora/python_packages/wofs_ml_severe/tests/test_data/wofs_30M_07_20210504_2205_2235.nc'
        track_file = file_30M.replace('30M', 'ENSEMBLETRACKS')
        generate_ensemble_track_file(file_30M)

        # Create fake data (NV, NE, NY, NX)
        ens_data = np.random.normal(size=(NV,NE,NY,NX))
        
        ds = {}
        for typ in ['ens', 'env', 'svr']:
            data = {f'{typ}_{i+1}' : (['NE', 'NY', 'NX'], ens_data[i,:,:,:]) for i in range(NV)}

            if typ == 'ens':
                data['uh_2to5'] = (['NE', 'NY', 'NX'], ens_data[0,:,:,:])
                data['ws_80'] = (['NE', 'NY', 'NX'], ens_data[0,:,:,:])
                data['hailcast'] = (['NE', 'NY', 'NX'], ens_data[0,:,:,:])
            
            ds[typ] = xr.Dataset(data)
            
        
        ens_filenames = [f'wofs_ENS_{i:02d}' for i in range(6)]
        env_filename = ['wofs_ENV_06']
        svr_filename = ['wofs_SVR_06']
        
        all_filenames = ens_filenames + env_filename + svr_filename
        
        files_to_load = [{'track_file' : track_file,
                         'ens_file' : ens_filenames,
                         'env_file' : env_filename[0],
                         'svr_file' : svr_filename[0],
                        }]
                         
        for f in all_filenames:
            typ = f.split('_')[1].lower()
            ds[typ].to_netcdf(f) 
        
        data_generator = MLDataGenerator(files_to_load, TEMP=False, 
                ml_config_path='/home/monte.flora/python_packages/wofs_ml_severe/tests/test_data/ml_config.yml')

        data_generator(runtype='rto') 
        
        # Checks that MLDATA file has been created. 
        # (optional) check the len(data) == n_objects. 
        self.assertIsFile(track_file.replace('ENSEMBLETRACKS', 'MLDATA').replace('.nc', '.feather'))

In [18]:
test_feature_extract()

Saving /home/monte.flora/python_packages/wofs_ml_severe/tests/test_data/wofs_ENSEMBLETRACKS_07_20210504_2205_2235.nc...
dict_keys(['ens', 'env', 'svr']) ens wofs_ENS_00
dict_keys(['ens', 'env', 'svr']) ens wofs_ENS_01
dict_keys(['ens', 'env', 'svr']) ens wofs_ENS_02
dict_keys(['ens', 'env', 'svr']) ens wofs_ENS_03
dict_keys(['ens', 'env', 'svr']) ens wofs_ENS_04
dict_keys(['ens', 'env', 'svr']) ens wofs_ENS_05
dict_keys(['ens', 'env', 'svr']) env wofs_ENV_06
dict_keys(['ens', 'env', 'svr']) svr wofs_SVR_06


KeyError: 'xlat'

### 3. Matching Storm Tracks to ML Features 

### 4. Concatenate into single dataframe 